In [3]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import ipyvuetify as v

In [5]:
# List all opponents reach, height, and W/L in a table

df_fight_results = pd.read_csv('data/github/ufc_fight_results.csv')
df_event_details = pd.read_csv('data/github/ufc_event_details.csv')
df_fighter_tott = pd.read_csv('data/github/ufc_fighter_tott.csv')

fighter = 'Dustin Poirier'

# Cleaning fighter names for better matching
df_fighter_tott['FIGHTER_CLEAN'] = df_fighter_tott['FIGHTER'].str.lower().str.replace(' ', '')

# Extracting the relevant fight results that include Dustin Poirier
fighter_bouts = df_fight_results[df_fight_results['BOUT'].str.contains(fighter, case=False)].copy()
fighter_bouts[['Fighter1', 'Fighter2']] = fighter_bouts['BOUT'].str.split(' vs. ', expand=True)
fighter_bouts[['Result_Fighter1', 'Result_Fighter2']] = fighter_bouts['OUTCOME'].str.split('/', expand=True)
fighter_bouts['Method'] = fighter_bouts['METHOD']  # Ensure METHOD is included correctly

# Determining the opponent name, Poirier's result, and method directly
opponents_results_df = pd.DataFrame()
opponents_results_df['Opponent'] = fighter_bouts.apply(lambda row: row['Fighter2'] if fighter in row['Fighter1'] else row['Fighter1'], axis=1)
opponents_results_df['Result'] = fighter_bouts.apply(lambda row: row['Result_Fighter1'] if fighter in row['Fighter1'] else row['Result_Fighter2'], axis=1)
opponents_results_df['Method'] = fighter_bouts['Method']
opponents_results_df['Opponent_Clean'] = opponents_results_df['Opponent'].str.lower().str.replace(' ', '')

# Merging opponent results with their stats from the fighter totals
opponents_stats_merged = opponents_results_df.merge(df_fighter_tott, left_on='Opponent_Clean', right_on='FIGHTER_CLEAN', how='left')

# Selecting the final columns for the DataFrame and including Method and Stance
opponents_stats_final = opponents_stats_merged[['Opponent', 'HEIGHT', 'REACH', 'Method', 'STANCE', 'Result']]
opponents_stats_final.columns = ['Opponent', 'Height', 'Reach', 'Method', 'Stance', 'Result']

# Create a copy of the DataFrame before dropping NA values
opponents_stats_final = opponents_stats_final.copy()

# Now drop rows with missing data in 'Height' and 'Reach' columns
opponents_stats_final.dropna(subset=['Height', 'Reach'], inplace=True)

# Resetting the index after dropping the NA values
opponents_stats_final.reset_index(drop=True, inplace=True)

# Save to csv (optional)
# opponents_stats_final.to_csv('./data/opponent_reach.csv', index=False)
opponents_stats_final.to_csv(f'./data/opponent_reach_{fighter.replace(" ", "_").lower()}.csv', index=False)

# Applying color to the 'Result' column based on the outcome
def color_results(val):
    if val == 'W':
        color = 'lightgreen'
    elif val == 'L':
        color = 'salmon'
    else:  # For 'NC' and anything else
        color = 'yellow'
    return f'background-color: {color}'

# Now apply the styling to the DataFrame
styled_df = opponents_stats_final.style.map(color_results, subset=['Result'])
styled_df

,Opponent,Height,Reach,Method,Stance,Result
0,Islam Makhachev,"5' 10""","70""",Submission,Southpaw,L
1,Benoit Saint Denis,"5' 11""","73""",KO/TKO,Southpaw,W
2,Justin Gaethje,"5' 11""","70""",KO/TKO,Orthodox,L
3,Michael Chandler,"5' 8""","71""",Submission,Orthodox,W
4,Charles Oliveira,"5' 10""","74""",Submission,Orthodox,L
5,Conor McGregor,"5' 9""","74""",TKO - Doctor's Stoppage,Southpaw,W
6,Conor McGregor,"5' 9""","74""",KO/TKO,Southpaw,W
7,Dan Hooker,"6' 0""","75""",Decision - Unanimous,Switch,W
8,Khabib Nurmagomedov,"5' 10""","70""",Submission,Orthodox,L
9,Max Holloway,"5' 11""","69""",Decision - Unanimous,Orthodox,W


In [ ]:
# In the welterweight division how often does the fighter with the longer reach win compared to lose?
# Give me the total w/l and detailed statistics of the trends.

In [9]:
# division = 'welterweight'

master_df = pd.read_csv('./data/github/master.csv')

weight_divisions = [
    'Heavyweight',
    'Light Heavyweight',
    'Middleweight',
    'Welterweight',
    'Lightweight',
    'Featherweight',
    'Bantamweight',
    'Flyweight'
]

placeholder = 'Select a division'
dropdown = v.Select(
    items=[{'text': placeholder, 'value': None}] + [{'text': division, 'value': division} for division in weight_divisions],
    label='Division',
    v_model=None
)

def on_change(widget, event, data):
    global division
    if widget.v_model is not None:
        division = widget.v_model
#         print(f"The selected division is: {division}")

dropdown.on_event('change', on_change)
display(dropdown)


Select(items=[{'text': 'Select a division', 'value': None}, {'text': 'Heavyweight', 'value': 'Heavyweight'}, {…

In [12]:
division_fights = master_df[master_df['WEIGHTCLASS'].str.contains(division, case=False, na=False)]

total_wins_longer_reach = 0
total_losses_longer_reach = 0

for _, row in division_fights.iterrows():
    fighter1 = row['FIGHTER1']
    fighter2 = row['FIGHTER2']
    reach1 = row['FIGHTER1_REACH']
    reach2 = row['FIGHTER2_REACH']
    winning_fighter = row['WINNING_FIGHTER']

    # Skip rows with missing or invalid reach data
    if pd.isna(reach1) or pd.isna(reach2) or (reach1 == '--') or (reach2 == '--'):
        continue

    reach1_value = float(reach1.replace('"', ''))
    reach2_value = float(reach2.replace('"', ''))

    if reach1_value > reach2_value:
        longer_reach_fighter = fighter1
    elif reach2_value > reach1_value:
        longer_reach_fighter = fighter2
    else:
        continue  # Skip if reach is equal

    if winning_fighter.lower() == longer_reach_fighter.lower():
        total_wins_longer_reach += 1
    else:
        total_losses_longer_reach += 1

print(f"Overall Win/Loss Record of Fighters with Longer Reach in the {division.capitalize()} Division:")
print(f"Total Wins: {total_wins_longer_reach}")
print(f"Total Losses: {total_losses_longer_reach}")


Overall Win/Loss Record of Fighters with Longer Reach in the Welterweight Division:
Total Wins: 452
Total Losses: 439


In [13]:
# Minimum 3 inch advantage 

division_fights = master_df[master_df['WEIGHTCLASS'].str.contains(division, case=False, na=False)]

total_wins_longer_reach = 0
total_losses_longer_reach = 0

for _, row in division_fights.iterrows():
    fighter1 = row['FIGHTER1']
    fighter2 = row['FIGHTER2']
    reach1 = row['FIGHTER1_REACH']
    reach2 = row['FIGHTER2_REACH']
    winning_fighter = row['WINNING_FIGHTER']

    if pd.isna(reach1) or pd.isna(reach2) or (reach1 == '--') or (reach2 == '--'):
        continue

    reach1_value = float(reach1.replace('"', ''))
    reach2_value = float(reach2.replace('"', ''))

    reach_difference = abs(reach1_value - reach2_value)
    
    if reach_difference < 3:
        continue

    if reach1_value > reach2_value:
        longer_reach_fighter = fighter1
    elif reach2_value > reach1_value:
        longer_reach_fighter = fighter2
    else:
        continue

    if winning_fighter.lower() == longer_reach_fighter.lower():
        total_wins_longer_reach += 1
    else:
        total_losses_longer_reach += 1

print(f"Overall Win/Loss Record of Fighters with Longer Reach (3 inches or greater) in the {division.capitalize()} Division:")
print(f"Total Wins: {total_wins_longer_reach}")
print(f"Total Losses: {total_losses_longer_reach}")


Overall Win/Loss Record of Fighters with Longer Reach (3 inches or greater) in the Welterweight Division:
Total Wins: 226
Total Losses: 223


In [14]:
# Minimum 5 inch advantages

division_fights = master_df[master_df['WEIGHTCLASS'].str.contains(division, case=False, na=False)]

total_wins_longer_reach = 0
total_losses_longer_reach = 0

for _, row in division_fights.iterrows():
    fighter1 = row['FIGHTER1']
    fighter2 = row['FIGHTER2']
    reach1 = row['FIGHTER1_REACH']
    reach2 = row['FIGHTER2_REACH']
    winning_fighter = row['WINNING_FIGHTER']

    if pd.isna(reach1) or pd.isna(reach2) or (reach1 == '--') or (reach2 == '--'):
        continue

    reach1_value = float(reach1.replace('"', ''))
    reach2_value = float(reach2.replace('"', ''))

    reach_difference = abs(reach1_value - reach2_value)
    
    if reach_difference < 5:
        continue

    if reach1_value > reach2_value:
        longer_reach_fighter = fighter1
    elif reach2_value > reach1_value:
        longer_reach_fighter = fighter2
    else:
        continue

    if winning_fighter.lower() == longer_reach_fighter.lower():
        total_wins_longer_reach += 1
    else:
        total_losses_longer_reach += 1

print(f"Overall Win/Loss Record of Fighters with Longer Reach (5 inches or greater) in the {division.capitalize()} Division:")
print(f"Total Wins: {total_wins_longer_reach}")
print(f"Total Losses: {total_losses_longer_reach}")

Overall Win/Loss Record of Fighters with Longer Reach (5 inches or greater) in the Welterweight Division:
Total Wins: 92
Total Losses: 73


In [ ]:
### Fancy buttons for them

In [ ]:
# Get unique weight classes
weight_classes = sorted(master_df['WEIGHTCLASS'].dropna().unique())

# Create radio buttons grouped in rows of 3
def create_radio_buttons(options, items_per_row=3):
    rows = []
    for i in range(0, len(options), items_per_row):
        row = widgets.HBox([widgets.RadioButtons(options=options[i:i+items_per_row], description='', layout=widgets.Layout(width='auto'))])
        rows.append(row)
    return widgets.VBox(rows)

# Create the radio buttons for weight class selection
radio_buttons = create_radio_buttons(weight_classes, items_per_row=3)

# Output widget to display results
output = widgets.Output()

def calculate_reach_advantage(change):
    with output:
        clear_output()
        division = change['new']
        
        # Filter the dataset for the specified division
        division_fights = master_df[master_df['WEIGHTCLASS'].str.contains(division, case=False, na=False)]
        
        # Initialize win/loss counters
        total_wins_longer_reach = 0
        total_losses_longer_reach = 0
        
        # Update win/loss records based on fight outcomes
        for _, row in division_fights.iterrows():
            fighter1 = row['FIGHTER1']
            fighter2 = row['FIGHTER2']
            reach1 = row['FIGHTER1_REACH']
            reach2 = row['FIGHTER2_REACH']
            winning_fighter = row['WINNING_FIGHTER']
        
            # Skip rows with missing or invalid reach data
            if pd.isna(reach1) or pd.isna(reach2) or (reach1 == '--') or (reach2 == '--'):
                continue
        
            reach1_value = float(reach1.replace('"', ''))
            reach2_value = float(reach2.replace('"', ''))
        
            if reach1_value > reach2_value:
                longer_reach_fighter = fighter1
            elif reach2_value > reach1_value:
                longer_reach_fighter = fighter2
            else:
                continue  # Skip if reach is equal
        
            if winning_fighter.lower() == longer_reach_fighter.lower():
                total_wins_longer_reach += 1
            else:
                total_losses_longer_reach += 1
        
        # Display the results
        print(f"Overall Win/Loss Record of Fighters with Longer Reach in the {division} Division:")
        print(f"Total Wins: {total_wins_longer_reach}")
        print(f"Total Losses: {total_losses_longer_reach}")

# Attach the calculation function to the radio buttons
for row in radio_buttons.children:
    for btn in row.children:
        btn.observe(lambda change: calculate_reach_advantage(change), names='value')

# Display the radio buttons and the output widget
display(radio_buttons, output)

In [ ]:
# import pandas as pd
# import ipywidgets as widgets
# from IPython.display import display, clear_output

# # Load the dataset
# master_df = pd.read_csv('/mnt/data/master.csv')

# Get unique weight classes
weight_classes = sorted(master_df['WEIGHTCLASS'].dropna().unique())

# Create a dropdown menu for weight class selection
dropdown = widgets.Dropdown(
    options=weight_classes,
    description='Division:',
    disabled=False,
)

# Output widget to display results
output = widgets.Output()

def calculate_reach_advantage(change):
    with output:
        clear_output()
        division = change['new']
        
        # Filter the dataset for the specified division
        division_fights = master_df[master_df['WEIGHTCLASS'].str.contains(division, case=False, na=False)]
        
        # Initialize win/loss counters
        total_wins_longer_reach = 0
        total_losses_longer_reach = 0
        
        # Update win/loss records based on fight outcomes
        for _, row in division_fights.iterrows():
            fighter1 = row['FIGHTER1']
            fighter2 = row['FIGHTER2']
            reach1 = row['FIGHTER1_REACH']
            reach2 = row['FIGHTER2_REACH']
            winning_fighter = row['WINNING_FIGHTER']
        
            # Skip rows with missing or invalid reach data
            if pd.isna(reach1) or pd.isna(reach2) or (reach1 == '--') or (reach2 == '--'):
                continue
        
            reach1_value = float(reach1.replace('"', ''))
            reach2_value = float(reach2.replace('"', ''))
        
            if reach1_value > reach2_value:
                longer_reach_fighter = fighter1
            elif reach2_value > reach1_value:
                longer_reach_fighter = fighter2
            else:
                continue  # Skip if reach is equal
        
            if winning_fighter.lower() == longer_reach_fighter.lower():
                total_wins_longer_reach += 1
            else:
                total_losses_longer_reach += 1
        
        # Display the results
        print(f"Overall Win/Loss Record of Fighters with Longer Reach in the {division} Division:")
        print(f"Total Wins: {total_wins_longer_reach}")
        print(f"Total Losses: {total_losses_longer_reach}")

# Attach the calculation function to the dropdown menu
dropdown.observe(calculate_reach_advantage, names='value')

# Display the dropdown menu and the output widget
display(dropdown, output)


In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load the dataset
# master_df = pd.read_csv('/mnt/data/master.csv')

# Normalize the weight classes to combine title bouts and normal bouts
master_df['WEIGHTCLASS'] = master_df['WEIGHTCLASS'].str.replace(' Title Bout', '', regex=False)
master_df['WEIGHTCLASS'] = master_df['WEIGHTCLASS'].str.replace(' Interim', '', regex=False)

# Get unique weight classes
weight_classes = sorted(master_df['WEIGHTCLASS'].dropna().unique())

# Create a dropdown menu for weight class selection
dropdown = widgets.Dropdown(
    options=weight_classes,
    description='Division:',
    disabled=False,
)

# Output widget to display results
output = widgets.Output()

def calculate_reach_advantage(change):
    with output:
        clear_output()
        division = change['new']
        
        # Filter the dataset for the specified division
        division_fights = master_df[master_df['WEIGHTCLASS'].str.contains(division, case=False, na=False)]
        
        # Initialize win/loss counters
        total_wins_longer_reach = 0
        total_losses_longer_reach = 0
        
        # Update win/loss records based on fight outcomes
        for _, row in division_fights.iterrows():
            fighter1 = row['FIGHTER1']
            fighter2 = row['FIGHTER2']
            reach1 = row['FIGHTER1_REACH']
            reach2 = row['FIGHTER2_REACH']
            winning_fighter = row['WINNING_FIGHTER']
        
            # Skip rows with missing or invalid reach data
            if pd.isna(reach1) or pd.isna(reach2) or (reach1 == '--') or (reach2 == '--'):
                continue
        
            reach1_value = float(reach1.replace('"', ''))
            reach2_value = float(reach2.replace('"', ''))
        
            if reach1_value > reach2_value:
                longer_reach_fighter = fighter1
            elif reach2_value > reach1_value:
                longer_reach_fighter = fighter2
            else:
                continue  # Skip if reach is equal
        
            if winning_fighter.lower() == longer_reach_fighter.lower():
                total_wins_longer_reach += 1
            else:
                total_losses_longer_reach += 1
        
        # Display the results
        print(f"Overall Win/Loss Record of Fighters with Longer Reach in the {division} Division:")
        print(f"Total Wins: {total_wins_longer_reach}")
        print(f"Total Losses: {total_losses_longer_reach}")

# Attach the calculation function to the dropdown menu
dropdown.observe(calculate_reach_advantage, names='value')

# Display the dropdown menu and the output widget
display(dropdown, output)


In [ ]:
import pandas as pd

def calculate_reach_advantage_wins_losses(division):
    # Load the datasets
    master_df = pd.read_csv('./data/github/master.csv')
    fighter_info_df = pd.read_csv('./data/github/ufc_fighter_tott.csv')

    # Normalize the FIGHTER names to match the cleaning process
    fighter_info_df['FIGHTER'] = fighter_info_df['FIGHTER'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)
    master_df['FIGHTER1'] = master_df['FIGHTER1'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)
    master_df['FIGHTER2'] = master_df['FIGHTER2'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)

    # Merge reach data for FIGHTER1 and FIGHTER2
    master_df = master_df.merge(fighter_info_df[['FIGHTER', 'REACH']], how='left', left_on='FIGHTER1', right_on='FIGHTER').rename(columns={'REACH': 'FIGHTER1_REACH'}).drop(columns=['FIGHTER'])
    master_df = master_df.merge(fighter_info_df[['FIGHTER', 'REACH']], how='left', left_on='FIGHTER2', right_on='FIGHTER').rename(columns={'REACH': 'FIGHTER2_REACH'}).drop(columns=['FIGHTER'])

    # Filter the dataset for the specified division
    division_fights = master_df[master_df['WEIGHTCLASS'].str.contains(division, case=False, na=False)]

    # Initialize dictionaries to keep track of win/loss records for fighters with longer reach
    fighter_wins_longer_reach = {}
    fighter_losses_longer_reach = {}

    # Update win/loss records based on fight outcomes
    for _, row in division_fights.iterrows():
        fighter1 = row['FIGHTER1']
        fighter2 = row['FIGHTER2']
        reach1 = row['FIGHTER1_REACH']
        reach2 = row['FIGHTER2_REACH']
        winning_fighter = row['WINNING_FIGHTER']

        # Skip rows with missing or invalid reach data
        if pd.isna(reach1) or pd.isna(reach2) or reach1 == '--' or reach2 == '--':
            continue

        reach1_value = float(reach1.replace('"', ''))
        reach2_value = float(reach2.replace('"', ''))

        if reach1_value > reach2_value:
            longer_reach_fighter = fighter1
        elif reach2_value > reach1_value:
            longer_reach_fighter = fighter2
        else:
            continue  # Skip if reach is equal

        if longer_reach_fighter not in fighter_wins_longer_reach:
            fighter_wins_longer_reach[longer_reach_fighter] = 0
            fighter_losses_longer_reach[longer_reach_fighter] = 0

        if winning_fighter.lower() == longer_reach_fighter.lower():
            fighter_wins_longer_reach[longer_reach_fighter] += 1
        else:
            fighter_losses_longer_reach[longer_reach_fighter] += 1

    # Calculate the total wins and losses for fighters with longer reach
    total_wins_longer_reach = sum(fighter_wins_longer_reach.values())
    total_losses_longer_reach = sum(fighter_losses_longer_reach.values())

    return total_wins_longer_reach, total_losses_longer_reach

# Input the division you want to research
division_input = input("Enter the division you want to research (e.g., welterweight): ").strip().lower()

# Calculate the results for the specified division
wins, losses = calculate_reach_advantage_wins_losses(division_input)

print(f"Overall Win/Loss Record of Fighters with Longer Reach in the {division_input.capitalize()} Division:")
print(f"Total Wins: {wins}")
print(f"Total Losses: {losses}")


In [ ]:
import pandas as pd

# Load the datasets
master_df = pd.read_csv('./data/github/master.csv')
fighter_info_df = pd.read_csv('./data/github/ufc_fighter_tott.csv')

# Normalize the FIGHTER names to match the cleaning process
fighter_info_df['FIGHTER'] = fighter_info_df['FIGHTER'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)
master_df['FIGHTER1'] = master_df['FIGHTER1'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)
master_df['FIGHTER2'] = master_df['FIGHTER2'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)

# Merge reach data for FIGHTER1 and FIGHTER2
master_df = master_df.merge(fighter_info_df[['FIGHTER', 'REACH']], how='left', left_on='FIGHTER1', right_on='FIGHTER').rename(columns={'REACH': 'FIGHTER1_REACH'}).drop(columns=['FIGHTER'])
master_df = master_df.merge(fighter_info_df[['FIGHTER', 'REACH']], how='left', left_on='FIGHTER2', right_on='FIGHTER').rename(columns={'REACH': 'FIGHTER2_REACH'}).drop(columns=['FIGHTER'])

# Save the updated master_df to the same file
master_df.to_csv('./data/github/master.csv', index=False)

# Input the division you want to research
division = input("Enter the division you want to research (e.g., welterweight): ").strip().lower()

# Filter the dataset for the specified division
division_fights = master_df[master_df['WEIGHTCLASS'].str.contains(division, case=False, na=False)]

# Initialize win/loss counters
total_wins_longer_reach = 0
total_losses_longer_reach = 0

# Update win/loss records based on fight outcomes
for _, row in division_fights.iterrows():
    fighter1 = row['FIGHTER1']
    fighter2 = row['FIGHTER2']
    reach1 = row['FIGHTER1_REACH']
    reach2 = row['FIGHTER2_REACH']
    winning_fighter = row['WINNING_FIGHTER']

    # Skip rows with missing or invalid reach data
    if pd.isna(reach1) or pd.isna(reach2) or reach1 == '--' or reach2 == '--':
        continue

    reach1_value = float(reach1.replace('"', ''))
    reach2_value = float(reach2.replace('"', ''))

    if reach1_value > reach2_value:
        longer_reach_fighter = fighter1
    elif reach2_value > reach1_value:
        longer_reach_fighter = fighter2
    else:
        continue  # Skip if reach is equal

    if winning_fighter.lower() == longer_reach_fighter.lower():
        total_wins_longer_reach += 1
    else:
        total_losses_longer_reach += 1

# Print the results
print(f"Overall Win/Loss Record of Fighters with Longer Reach in the {division.capitalize()} Division:")
print(f"Total Wins: {total_wins_longer_reach}")
print(f"Total Losses: {total_losses_longer_reach}")


In [ ]:
# Input the division you want to research
division = input("Enter the division you want to research (e.g., welterweight): ").strip().lower()

# Load the datasets
master_df = pd.read_csv('./data/github/master.csv')
fighter_info_df = pd.read_csv('./data/github/ufc_fighter_tott.csv')

# Normalize the FIGHTER names to match the cleaning process
fighter_info_df['FIGHTER'] = fighter_info_df['FIGHTER'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)
master_df['FIGHTER1'] = master_df['FIGHTER1'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)
master_df['FIGHTER2'] = master_df['FIGHTER2'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)

# Merge reach data for FIGHTER1 and FIGHTER2
master_df = master_df.merge(fighter_info_df[['FIGHTER', 'REACH']], how='left', left_on='FIGHTER1', right_on='FIGHTER').rename(columns={'REACH': 'FIGHTER1_REACH'}).drop(columns=['FIGHTER'])
master_df = master_df.merge(fighter_info_df[['FIGHTER', 'REACH']], how='left', left_on='FIGHTER2', right_on='FIGHTER').rename(columns={'REACH': 'FIGHTER2_REACH'}).drop(columns=['FIGHTER'])

# Save the updated master_df to the same file
master_df.to_csv('./data/github/master.csv', index=False)

# Filter the dataset for the specified division
division_fights = master_df[master_df['WEIGHTCLASS'].str.contains(division, case=False, na=False)]

# Initialize win/loss counters
total_wins_longer_reach = 0
total_losses_longer_reach = 0

# Update win/loss records based on fight outcomes
for _, row in division_fights.iterrows():
    fighter1 = row['FIGHTER1']
    fighter2 = row['FIGHTER2']
    reach1 = row['FIGHTER1_REACH']
    reach2 = row['FIGHTER2_REACH']
    winning_fighter = row['WINNING_FIGHTER']

    # Skip rows with missing or invalid reach data
    if pd.isna(reach1) or pd.isna(reach2) or reach1 == '--' or reach2 == '--':
        continue

    reach1_value = float(reach1.replace('"', ''))
    reach2_value = float(reach2.replace('"', ''))

    if reach1_value > reach2_value:
        longer_reach_fighter = fighter1
    elif reach2_value > reach1_value:
        longer_reach_fighter = fighter2
    else:
        continue  # Skip if reach is equal

    if winning_fighter.lower() == longer_reach_fighter.lower():
        total_wins_longer_reach += 1
    else:
        total_losses_longer_reach += 1

# Print the results
print(f"Overall Win/Loss Record of Fighters with Longer Reach in the {division.capitalize()} Division:")
print(f"Total Wins: {total_wins_longer_reach}")
print(f"Total Losses: {total_losses_longer_reach}")


In [ ]:
# Toggle Buttons

# toggle_buttons = widgets.ToggleButtons(
#     options=weight_divisions,
#     description='Select Division:',
#     disabled=False,
# )

# display(toggle_buttons)
# toggle_buttons.observe(lambda change: print(f"Selected Division: {toggle_buttons.value}"), names='value')

In [ ]:
# import ipywidgets as widgets
# from IPython.display import display

# # Define weight divisions for MMA
# weight_divisions = [
#     'Heavyweight',
#     'Light Heavyweight',
#     'Middleweight',
#     'Welterweight',
#     'Lightweight',
#     'Featherweight',
#     'Bantamweight',
#     'Flyweight',
#     'Strawweight'
# ]

# dropdown = widgets.Dropdown(
#     options=weight_divisions,
#     description='Select Division:',
#     disabled=False,
# )

# display(dropdown)
# division = None

# def on_select(change):
#     global division
#     if change['new'] != change['old']:  # Only update if the value has actually changed
#         division = change['new']
#         print(f"The selected division is: {division}")

# dropdown.observe(on_select, names='value')
